## Markov Chain Monte Carlo and Animation using Matplotlib

Let us say we make n (say 15) observations of an event (employee height), which is mapped to a number (Random Variable). Our belief (The prior) that the average value of follows a normal distribution that has a mean of 70 and standard deviation is 6. We have no knowledge of the distribution that generated the data (The posterior)

The requirement is, say, to generate 1000 samples from the unknown posterior distribution.

Markov Chain Monte Carlo enables sample generation from the posterior distribution, without knowing the parameters of the posterior distribution.

MCMC constructs series of 'steps', where each step represents an update to the belies about the distribution the observed data is from. The steps are constructed such that the belief doesn't 'converge' to a single value. However, if you look at the chain of steps themselves, you may be able to observe that a (large enough) sequence of k steps is similar to another k step sequence in its characteristiics. Each of these k steps then represents a sample from the unknown posterior distribution.

There are three steps to generating 1 sample and and entry/input condition:
Input condition: you must have a current belief or guess about the unknown (posterior) distribution.

1. Update the current guess to a 'proposed' guess using a proposal mechanism. 
The proposal mechanism is a way to perturb the current guess to a different value, using a proposal distribution. The proposal is then tested using the acceptance mechanism.

Below, we sample mean from a normal distribution and sigma from an inverse gamma distribution. A later version of the notebook will talk more about the sigma.

The choice of the proposal distribution has some implication on how easy is it to generate samples. Remember, by perturbing the current guess, we have only generated a proposal for the parameter, we have not declared it to an official sample yet. If our proposal mechanism generates guesses that always move 'far' from the current guess, the acceptance rate may be low. On the other hand,guesses very near to the current guess may mean that the MCMC may take 'too long' to converge.

2. Determine the likelihood of observing the data with current guess and with proposal
When a proposal is generated in step 1, we need to check if it explains the observed data (i.e. our 15 height observations) any better than the current guess. How do we measure the 'better' or 'worse' amd how do we quantify it? 

We need to use a 'likelihood' function, or at least a version that may be proportional to the likelihood function. What is likelihood? I will take a couple of lines to explain, more knowledgeable practistioners can choose to skip.

Likelihood function is just what it says it is. Likelihood(data) is the probability that "exactly this data" will be observed, given a probability distribution. 

This is how you calculate likelihood function:
a. Calculate probability of observing each portion of data separately
b. Assuming independence betwen all the observations, multiply all the probabilitoes together. That is it.

Note: Ideally, log likelihoods should be used to reduce computational inaccuracies as likelihoods could be very small numbers.


Recall that our current guess and the proposal guess represent a parameter for the probability function used in the above calculation. Therefore the likelihood will be different as the guessed parameter changes.

3. Proposal Acceptance/Rejection
If the likelihood of the data (being observed) increases with tne new guess, we accept the guess.

If the likelihood of the data (being observed) decreases with tne new guess, we sometimes accept the guess and sometimes accept the guess. A good way to think is, we want to accept worse outcomes with lower probability than less worse outcomes.

so our acceptance function looks like 
    p_accept = min(1, (likelihood(data) with proposal)/(likelihood(data) with current guess))

If the proposal is accepted, the proposal is added as a sample from mcmc; the proposal now becomes the current guess and the you can go back to step 1 to generate the next sample.

#### Why does this work?
Why do this series of steps work to generate a sample from the true posterior distribution? After all, we don't see a convergence to the 'actual' distribution. The proposals just seem to be bouncing around! also, how can the proposal be a sample itself?

The answer is, with each mcmc step, the acceptance algorithm (Called the Metropolis-Hastings sampler) tries to push towards a distribution that explains the data better, but balances that against the need for exploring the sample space. Since we don't know the true posterior distribution, only accepting proposals that explain the data better wcould move the proposal towards a local convergence, which may lead to biased samples. The exploratory part of the sampler aceepts proposals that worsen the likelihood, but with a lower probability. Here, you can see that the acceptance function is shaping the sample space based on the explanatory power of the sample. One can see that proposal means with lower explanatory power are further from a 'true' mean and have a lower chance of being selected as a sample. This fits in well with the idea that if draws were being done from the true posterior, samples that explain the data better have a higher chance of being drawn and vice-versa.

#### Does my initial guess matter?
Ideally, no. Practically, maybe. If the initial guess is far away from the 'true' value of the parameter, the mcmc may take a longer time to 'converge'. However, it is expected to converge after a while. Since we don't know what convergence will exactly look like, it may be practial to throw away first few (10?, 1000?, 10000?) and take the ones after that


#### Does it matter how far the proposal is from the current guess?
The width of the jump is a tuning question, balancing between quicker converge versus the acceptance probability for a proposals. Ability to make a bigger jump may at times lead to faster convergence, but may lower the ratio of acceptances to proposals made.



Below is an example of animated plots using matplotlib for mcmc model based on bayesian updates.

This is created using python 3.6
Step 1: import all required packages

In [245]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as ss
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from scipy.stats import norm, invgamma
import seaborn as sns
import sys

A helper function to determine if the code is running in a notebook or on a terminal

In [246]:
def ipython_info():
    ip = False
    if 'ipykernel' in sys.modules:
        ip = 'notebook'
    elif 'IPython' in sys.modules:
        ip = 'terminal'
    return ip=='notebook'

Step 1: Provide a prior. This is your guess on the mu and sigma

Step 2: You need to move the mu (and sigma) using the proposal distributions for mu and sigma


Step 3: Calculate the likelihood of the data with the prior estimates of x

Step 4: Calculate the likelihood of the data with the new estimates of the mu and sigma, estimated by moving from the previous step.

Step 5: Calculate Probability of the data * probability of the prior, using the prior mu and sigma (P(x/theta) * P(theta)

Step 6: Calculate Probability of data * probability of the mu_updated, using the new mu and sigma (P(x/thetanew) * P(thetanew)

Step 7: Create acceptance where a better move is always acceptable, a bad move is sometimes acceptable, with a bad move more acceptable than a worse move


For Later

Plug in an alternative to Metropolis algorithm and see what happens

Step 2: Create a bayesian update function
    This step creates a

In [247]:
def normal_mu_posterior_given_sigma(data, mu_0, sigma_0):
    n = len(data)
    sigma = invgamma.rvs(a=4.07, loc=0, scale=1, size=1, random_state=None)
    mu_post = (mu_0/sigma_0**2 + data.sum()/sigma**2)/(1.0/sigma_0**2 + n/sigma**2)
    sigma_post = 1.0/(1.0/ sigma_0**2 + n/sigma**2)
    return norm(mu_post, np.sqrt(sigma_post))

def ln_likelihood(mu_in,sigma_in,input_data,i):
    return np.sum(np.log(norm(mu_in,sigma_in).pdf(input_data)))

def likelihood(mu_in,sigma_in,input_data,i):
    return np.exp(ln_likelihood(mu_in,sigma_in,input_data,i))

def accept(mu_current,sigma_current,mu_proposal,sigma_proposal,data,i):
    # P(data given mu_current and sigma_current)*P(mu_current given initial prior)
    data_likely_mu_current=likelihood(mu_current, sigma_current,data,i)
    mu_current_likely_prior=likelihood(mu_prior_init, sd_prior_init,mu_current,i)
    # P(data given mu_proposal and sigma_proposal)*P(mu_proposal given initial prior)
    data_likely_mu_proposal = likelihood(mu_proposal, sigma_proposal,data,i)
    mu_proposal_likely_prior = likelihood(mu_prior_init, sd_prior_init,mu_proposal,i)
    # acceptance/rejection algorithm
    accept_sum= (data_likely_mu_proposal/data_likely_mu_current) * (mu_proposal_likely_prior / mu_current_likely_prior)
    if (norm(0,1).rvs() <accept_sum):
        accepted= True
        mu_current = mu_proposal
        sigma_current = sigma_proposal
    else:
        accepted =False

    
def update_dist(i):
    # We need to create global variables as the variables will be accesses and updated from more than one function
    #We want to view the path that our updated guesses follow. These are stored in the posterior
    posterior.append(mu_current)
    # This is the mechanism to create a new guess. We take the current value of the mean and use it in a normal distribution
    # to generate a new proposed value.
    # The move_control acts as 
    mu_proposal = norm(mu_current, move_control).rvs()
    # sigma_proposal = norm(sigma_current, 0.2*sigma_current).rvs()
    sigma_proposal = invgamma.rvs(a=4.07, loc=0, scale=1, size=1, random_state=None)
    accept(mu_current,sigma_current,mu_proposal,sigma_proposal,data,i)



In [248]:
x = np.linspace(-3, 3, 100)
data = np.random.randn(len(x))
move_control=.4
#The prior
mu_prior_init=0.5
sd_prior_init=1.2
prior=norm(mu_prior_init, sd_prior_init).pdf(x)
# Initiql guess is same as prior, cold start to the algo
mu_current = mu_prior_init
sigma_current = sd_prior_init
mu_proposal = mu_current
sigma_proposal = sigma_current
# mu_prior_mu=0
# mu_prior_sd=1
posterior=[mu_current]
accepted_posterior=[mu_current]
accepted=True
color = 'g' if accepted else 'r'

In [249]:
data_likely_mu_current=likelihood(mu_current, sigma_current,data,1)
mu_current_likely_prior=likelihood(mu_prior_init, sd_prior_init,mu_current,1)
# P(data given mu_proposal and sigma_proposal)*P(mu_proposal given initial prior)
data_likely_mu_proposal = likelihood(mu_proposal, sigma_proposal,data,1)
mu_proposal_likely_prior = likelihood(mu_prior_init, sd_prior_init,mu_proposal,1)
posterior_analytical = normal_mu_posterior_given_sigma(data, mu_prior_init, sd_prior_init)

In [250]:
fig, ((ax1, ax2), (ax3, ax4),(ax5,ax6)) = plt.subplots(3,2, figsize=(8,12))
# fig.figsize=(12,12)
fig.suptitle('Iteration %i' % (1))

#First figure - plots the prior distribution and shows change to proposed mean. The curve is fixed and teh vertical lines change
ax1.plot(x,prior) # The prior curve is plotted
ax1.set_xlim(-3, 3)
ax1.set_ylim(0, 0.42)

c1_curr_mu, = ax1.plot([], [], lw=2, linestyle='--',color='b') # vertical line represents current mean
# proposed \mu ,vertical line represents proposed mean, color will be green if accepted, else red
c1_prop_mu, = ax1.plot([], [], lw=2,marker='o',linestyle=':', color=color)

In [251]:
# Second figure. Changing distribution as the proposed mean is accepted and 
ax2.set_xlim(-3, 3)
ax2.set_ylim(0, 1.0)
sns.distplot(data, kde=False, norm_hist=True, ax=ax2)
c2, = ax2.plot([], [], lw=2)
c2_curr_mu, = ax2.plot([], [],color='b', linestyle='--', label='mu_current')
c2_prop_mu, = ax2.plot([],[], color=color, linestyle='--', label='mu_proposal')

In [252]:
# Third Figure - Show the analytical closed form posterior and how the mcmc samples look like
ax3.set_xlim(-3, 3)
ax3.set_ylim(0, 2.3)
ax3.plot(x,posterior_analytical.pdf(x))

# Fourth Figure - Plotting the path of mu_current
iterlength=300
ax4.set_xlim(0, iterlength)
ax4.set_ylim(-3,3)
c4, = ax4.plot([],[], lw=2)


In [253]:
# Figure 5 creates histogram of accepted mu_currrent values against the analytically calculated posterior with a fixed sigma
ax5.set_xlim(-3, 3)
ax5.set_ylim(0, 2.3)
ax5.plot(x,posterior_analytical.pdf(x))
ax6.set_xlim(0, iterlength)
ax6.set_ylim(-3,3)
c6, = ax6.plot([],[], lw=2)
plt.subplots_adjust(top=0.8)

In [254]:
def init():
    posterior.append(mu_current)
    accepted_posterior.append(mu_current)
    return c1_curr_mu, c1_prop_mu,c2,c2_curr_mu, c2_prop_mu,c4,c6

In [ ]:
def plot_proposal(i):
    global x, data, accepted, mu_current_likely_prior, mu_proposal_likely_prior,iterlength
    global posterior, accepted_posterior
    global mu_current, mu_proposal, sigma_current, sigma_proposal
    global data_likely_mu_current, data_likely_mu_proposal
    # Plot prior
    # update_dist(i)
    ########################
#     posterior.append(mu_current)
    # This is the mechanism to create a new guess. We take the current value of the mean and use it in a normal distribution
    # to generate a new proposed value.
    # The move_control controls how far the proposal jumps. 
    mu_proposal = norm(mu_current, move_control).rvs()
    # For sigma_proposal, note that the proposed jump is not taken from an inverse gamma distribution
    sigma_proposal = norm(sigma_current, 0.2*sigma_current).rvs()
    #accept(mu_current,sigma_current,mu_proposal,sigma_proposal,data,i)
    ############################################################
        # P(data given mu_current and sigma_current)*P(mu_current given initial prior)
    data_likely_mu_current=likelihood(mu_current, sigma_current,data,i)
    mu_current_likely_prior=likelihood(mu_prior_init, sd_prior_init,mu_current,i)
    # P(data given mu_proposal and sigma_proposal)*P(mu_proposal given initial prior)
    data_likely_mu_proposal = likelihood(mu_proposal, sigma_proposal,data,i)
    mu_proposal_likely_prior = likelihood(mu_prior_init, sd_prior_init,mu_proposal,i)
    # acceptance/rejection algorithm
    accept_sum= (data_likely_mu_proposal/data_likely_mu_current) * (mu_proposal_likely_prior / mu_current_likely_prior)
    if (norm(0,1).rvs() <accept_sum):
        accepted= True
        mu_current = mu_proposal
        sigma_current = sigma_proposal
    else:
        accepted =False
    ###############################################
    fig.suptitle('Iteration %i' % (i))
    c1_curr_mu.set_data([mu_current] * 2, [0, mu_current_likely_prior]) 
    
    # line showing probability of the current guess
    c1_prop_mu.set_data([mu_proposal] * 2, [0, mu_proposal_likely_prior])
    ax1.annotate("", xy=(mu_proposal, 0.2), xytext=(mu_current, 0.2))
    ax1.set(ylabel='Probability Density', title='mu_current=%.2f \nmu_current_likely_prior= %.2f\nmu_proposal=%.2f \nmu_proposal_likely_prior = %.2f' % 
            (mu_current, mu_current_likely_prior, mu_proposal, mu_proposal_likely_prior))
# set up the second curve
    y = norm(loc=mu_proposal, scale=sigma_proposal).pdf(x)
    c2.set_data(x, y)

    c2_curr_mu.set_data([mu_current] * 2, [0, 1.0])
    c2_prop_mu.set_data([mu_proposal] * 2, [0, 1.0])

#     ax2.annotate("", xy=(mu_proposal, 0.2), xytext=(mu_current, 0.2),
#                  arrowprops=dict(arrowstyle="->", lw=2.))
    ax2.set(title='Likelihood of mu_proposal|mu_current \nmu_current=%.2f\nlikelihood = %.2f\nmu_proposal=%.2f \nlikelihood = %.2f' % 
            (mu_current, data_likely_mu_current, mu_proposal, data_likely_mu_proposal))
    

    if accepted:
        # Update position
        mu_current = mu_proposal
        sigma_current = sigma_proposal
        accepted_posterior.append(mu_current)
    color = 'g' if accepted else 'r'    
    posterior.append(mu_current) # We will append even if the proposal is not accepted
    
    
    ax3.clear() # clearing the histogram before redrawing it
    ax3.set_xlim(-3, 3)
    ax3.set_ylim(0, 2.3)
    ax3.plot(x,posterior_analytical.pdf(x))
    ax3.set(title = 'Histogram: \mu proposal \n against the bayesian calculated posterior')
    sns.distplot(posterior, kde=False, norm_hist=True, ax=ax3)
    # Set curve 4 as the plot of the path of mu_current with every iteration
    ax4.set(title = 'Path of \mu proposal')
    c4.set_data(np.arange(len(posterior)),posterior)
 ##################
    ax5.clear() # clearing the histogram before redrawing it)
    ax5.set_xlim(-3, 3)
    ax5.set_ylim(0, 2.3)
    ax5.plot(x,posterior_analytical.pdf(x))
    ax5.set(title='Histogram: \mu current \n against the bayesian calculated posterior')
    sns.distplot(accepted_posterior, kde=False, norm_hist=True, ax=ax5)
    # Set curve 4 as the plot of the path of mu_current with every iteration
    ax6.set(title='Path of \mu current')
    c6.set_data(np.arange(len(accepted_posterior)),accepted_posterior)
    plt.subplots_adjust(top=0.8, left = 0.1)
    return c1_curr_mu, c1_prop_mu,c2,c2_curr_mu, c2_prop_mu,c4,c6


In [ ]:
anim = FuncAnimation(fig, plot_proposal, init_func=init, 
                               frames=iterlength, interval=200, blit=True)
HTML(anim.to_html5_video()) #if ipython_info() else plt.show()

C:\Users\ankur\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in double_scalars
